# Modelling NIQS Hardware pyTorch

In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import torch.optim as optim

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm


from src_torch import *

torch.set_printoptions(precision=8)

## Test

In [2]:
n = 3
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
torch.manual_seed(42)

X_target, _, _ = generate_ginibre(d**2, 2)

choi_target = generate_choi(X_target)

state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]


X, A, B = generate_ginibre(d**2, 2, requires_grad=True)
optimizer = optim.Adam([A, B], lr=0.01)

fid_list = []

In [ ]:
for i in tqdm(range(10000)):
    optimizer.zero_grad()
    X = A + 1j*B
    choi_model = generate_choi(X)
    index = np.random.randint(0, len(state_input_list)-1)
    state_input = state_input_list[index]
    state_target = state_target_list[index]
    
    state_model = apply_map(state_input, choi_model)
    fid = np.abs(state_fidelity(state_model, state_target).detach().numpy())
    loss = torch.norm(state_model - state_target)
    loss.backward()
    optimizer.step()
    fid_list.append(fid)
    fidelity = state_fidelity
    print(f"step: {i}, fid: {fid:.4f}, norm: {loss.detach().numpy():.4f}")

In [ ]:
plt.plot(fid_list)

In [4]:
print(choi_model)
print(choi_target)

tensor([[ 0.03807269-7.80625564e-18j, -0.01845127-4.90851245e-02j,
         -0.05548101-9.20165183e-03j,  ...,
         -0.02541757+1.30500987e-02j, -0.03580205+2.84096706e-02j,
         -0.08998313+2.88232404e-02j],
        [-0.01845127+4.90851245e-02j,  0.08739023-8.45677695e-18j,
          0.06797071-7.26427153e-02j,  ...,
          0.05556582-5.04095075e-02j,  0.00535633-9.13056476e-02j,
         -0.01680889-1.35073369e-01j],
        [-0.05548101+9.20165183e-03j,  0.06797071+7.26427153e-02j,
          0.14141996-1.38777878e-17j,  ...,
          0.15378825-2.48849924e-02j,  0.10429895-1.01460473e-01j,
          0.08122187-8.21124852e-02j],
        ...,
        [-0.02541757-1.30500987e-02j,  0.05556582+5.04095075e-02j,
          0.15378825+2.48849924e-02j,  ...,
          0.26784321-6.50521303e-18j,  0.15462760-1.12616032e-01j,
         -0.01837281-2.59314602e-02j],
        [-0.03580205-2.84096706e-02j,  0.00535633+9.13056476e-02j,
          0.10429895+1.01460473e-01j,  ...,
        

In [ ]:
A = state_input_list[15]
print(A)

L, V = torch.linalg.eigh(A)
L = torch.sqrt(L.type(torch.complex64))

B = torch.zeros_like(A)
for l, v in zip(L, V.T):
    B += l*torch.conj(v.reshape(-1,1))@v.reshape(1,-1)

print(A, B@B)

In [ ]:
n = 2
d = 2**n

np.random.seed(42)
X = generate_ginibre(d**2, 2)
print(X)

In [ ]:
XX = X@X.T.conj()
print(XX)

In [ ]:
[[ 0.78061957+0.j          0.07601179+0.68805429j -1.23559901-0.40182879j
   1.18303852+0.45402626j]
 [ 0.07601179-0.68805429j  3.17078055+0.j          0.27068591+2.97482211j
   3.25289552+0.88433786j]
 [-1.23559901+0.40182879j  0.27068591-2.97482211j  3.82883495+0.j
   0.10898574-1.62175541j]
 [ 1.18303852-0.45402626j  3.25289552-0.88433786j  0.10898574+1.62175541j
   6.37437797+0.j        ]]

In [ ]:
Y = partial_trace(XX)
print(Y)

In [ ]:
Y = square_root_inverse(Y)
print(Y)

In [ ]:
0.46982564+1.52046633e-19j
-0.00539533+2.72307567e-02j
-0.00539533-2.72307567e-02j
0.3258792 -1.16520961e-17j

In [ ]:
I = torch.eye(d).type(torch.complex128)
Ykron = torch.kron(I, Y).T
print(Ykron)

In [ ]:
choi = Ykron@XX@Ykron
print(torch.trace(choi))

In [30]:
n = 6
d = 2**n

rank = d**2

np.random.seed(42)
X, A, B = generate_ginibre(rank*d, d, requires_grad=True)
optimizer = optim.Adam([A, B], lr=0.01)

In [31]:
U, D, _ = torch.svd(X)
print(U.shape)
K = [U[i*d:(i+1)*d, :d] for i in range(rank)]

I = K[0].T.conj()@K[0] + K[1].T.conj()@K[1] + K[2].T.conj()@K[2] + K[3].T.conj()@K[3]
print(I)

torch.Size([262144, 64])
tensor([[ 9.84445812e-04+0.00000000e+00j, -1.88895202e-05-3.00589453e-05j,
         -1.67161390e-05+2.91756523e-05j,  ...,
          3.34065134e-05-4.28406897e-06j,  3.21246322e-06-1.14678946e-05j,
         -2.26206844e-07+7.45552601e-05j],
        [-1.88895202e-05+3.00589453e-05j,  8.82433347e-04+0.00000000e+00j,
         -4.17848200e-05+4.67988951e-05j,  ...,
          7.10125854e-05+2.88255037e-06j,  1.74818108e-05+2.48267567e-05j,
          4.94567582e-05+4.92595726e-05j],
        [-1.67161390e-05-2.91756523e-05j, -4.17848200e-05-4.67988951e-05j,
          1.04879530e-03+0.00000000e+00j,  ...,
         -8.67924494e-05-1.90293428e-05j, -8.46423041e-05-1.96070972e-05j,
         -3.30675121e-06+3.93717263e-05j],
        ...,
        [ 3.34065134e-05+4.28406897e-06j,  7.10125854e-05-2.88255037e-06j,
         -8.67924494e-05+1.90293428e-05j,  ...,
          1.03522385e-03+0.00000000e+00j,  9.22356606e-06-2.16760632e-05j,
         -1.38209246e-05+3.01280949e-05j]

In [32]:
eigen = D.detach().numpy()

min_val = 1e12

for i in eigen:
    for j in eigen:
        if i != j:
            new_min_val = np.abs((i**2 - j**2))
            if new_min_val < min_val:
                min_val = new_min_val

print(1/min_val)

0.008143678667540663


In [30]:
n = 2
d = 2**n
rank = 2

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
torch.manual_seed(42)

X_target, _, _ = generate_ginibre(rank*d, d)
kraus_list_target = generate_kraus(X_target, d, rank)

state_target_list = [apply_kraus(state_input, kraus_list_target) for state_input in state_input_list]


X, A, B = generate_ginibre(rank*d, d, requires_grad=True)
optimizer = optim.Adam([A, B], lr=0.01)

In [32]:
fid_list = []

for i in tqdm(range(10000)):
    optimizer.zero_grad()
    X = A + 1j*B
    kraus_list_model = generate_kraus(X, d, rank)
    index = np.random.randint(0, len(state_input_list)-1)
    state_input = state_input_list[index]
    state_target = state_target_list[index]
    
    state_model = apply_kraus(state_input, kraus_list_model)
    #loss = torch.norm(state_model - state_target)
    loss = -state_fidelity(state_model, state_target)
    loss.backward()
    optimizer.step()
    print(f"step: {i}, fid: {loss.detach().numpy():.4f}")
    

step: 0, fid: -0.6602-0.0000j
step: 1, fid: -0.9021-0.0000j
step: 2, fid: -0.9036-0.0000j
step: 3, fid: -0.9036+0.0000j
step: 4, fid: -0.7744+0.0000j
step: 5, fid: -0.9143+0.0000j
step: 6, fid: -0.7376-0.0000j
step: 7, fid: -0.9153-0.0000j
step: 8, fid: -0.7401+0.0000j
step: 9, fid: -0.7283-0.0000j
step: 10, fid: -0.7908-0.0000j
step: 11, fid: -0.6334-0.0000j
step: 12, fid: -0.9302+0.0000j
step: 13, fid: -0.8894-0.0000j
step: 14, fid: -0.7548+0.0000j
step: 15, fid: -0.8341-0.0000j
step: 16, fid: -0.8415-0.0000j
step: 17, fid: -0.9233+0.0000j
step: 18, fid: -0.6389+0.0000j
step: 19, fid: -0.9181+0.0000j
step: 20, fid: -0.6301+0.0000j
step: 21, fid: -0.7297-0.0000j
step: 22, fid: -0.6817+0.0000j
step: 23, fid: -0.8428+0.0000j
step: 24, fid: -0.8430-0.0000j
step: 25, fid: -0.9282-0.0000j
step: 26, fid: -0.6947+0.0000j
step: 27, fid: -0.8377-0.0000j
step: 28, fid: -0.6430+0.0000j
step: 29, fid: -0.7668+0.0000j
step: 30, fid: -0.9314+0.0000j
step: 31, fid: -0.8450-0.0000j
step: 32, fid: -0.

step: 336, fid: -0.9218-0.0000j
step: 337, fid: -0.8392-0.0000j
step: 338, fid: -0.5758+0.0000j
step: 339, fid: -0.9164+0.0000j
step: 340, fid: -0.7739-0.0000j
step: 341, fid: -0.8138-0.0000j
step: 342, fid: -0.9015-0.0000j
step: 343, fid: -0.8425-0.0000j
step: 344, fid: -0.9213+0.0000j
step: 345, fid: -0.8192-0.0000j
step: 346, fid: -0.9207+0.0000j
step: 347, fid: -0.8487+0.0000j
step: 348, fid: -0.9219+0.0000j
step: 349, fid: -0.5015+0.0000j
step: 350, fid: -0.8109+0.0000j
step: 351, fid: -0.8324+0.0000j
step: 352, fid: -0.8518-0.0000j
step: 353, fid: -0.9567-0.0000j
step: 354, fid: -0.8508-0.0000j
step: 355, fid: -0.8417-0.0000j
step: 356, fid: -0.9413-0.0000j
step: 357, fid: -0.7515+0.0000j
step: 358, fid: -0.9346-0.0000j
step: 359, fid: -0.7039+0.0000j
step: 360, fid: -0.9451+0.0000j
step: 361, fid: -0.9080+0.0000j
step: 362, fid: -0.6629-0.0000j
step: 363, fid: -0.8497-0.0000j
step: 364, fid: -0.8590+0.0000j
step: 365, fid: -0.9021-0.0000j
step: 366, fid: -0.7797-0.0000j
step: 36

step: 660, fid: -0.6769+0.0000j
step: 661, fid: -0.7593-0.0000j
step: 662, fid: -0.9354+0.0000j
step: 663, fid: -0.6990-0.0000j
step: 664, fid: -0.9364-0.0000j
step: 665, fid: -0.7635-0.0000j
step: 666, fid: -0.8281-0.0000j
step: 667, fid: -0.8928-0.0000j
step: 668, fid: -0.9151+0.0000j
step: 669, fid: -0.8299-0.0000j
step: 670, fid: -0.9179+0.0000j
step: 671, fid: -0.8524-0.0000j
step: 672, fid: -0.9428-0.0000j
step: 673, fid: -0.8544+0.0000j
step: 674, fid: -0.7411+0.0000j
step: 675, fid: -0.8691+0.0000j
step: 676, fid: -0.8994+0.0000j
step: 677, fid: -0.8938-0.0000j
step: 678, fid: -0.8088+0.0000j
step: 679, fid: -0.7061+0.0000j
step: 680, fid: -0.7755-0.0000j
step: 681, fid: -0.9445-0.0000j
step: 682, fid: -0.7236-0.0000j
step: 683, fid: -0.8444-0.0000j
step: 684, fid: -0.5353-0.0000j
step: 685, fid: -0.9184+0.0000j
step: 686, fid: -0.7142-0.0000j
step: 687, fid: -0.6511-0.0000j
step: 688, fid: -0.6780+0.0000j
step: 689, fid: -0.8626-0.0000j
step: 690, fid: -0.7646-0.0000j
step: 69

step: 994, fid: -0.9162+0.0000j
step: 995, fid: -0.7657-0.0000j
step: 996, fid: -0.9402+0.0000j
step: 997, fid: -0.8814+0.0000j
step: 998, fid: -0.9252-0.0000j
step: 999, fid: -0.8996+0.0000j
step: 1000, fid: -0.8357+0.0000j
step: 1001, fid: -0.5959+0.0000j
step: 1002, fid: -0.8423-0.0000j
step: 1003, fid: -0.8101-0.0000j
step: 1004, fid: -0.9272-0.0000j
step: 1005, fid: -0.6929+0.0000j
step: 1006, fid: -0.9184-0.0000j
step: 1007, fid: -0.8163-0.0000j
step: 1008, fid: -0.8189+0.0000j
step: 1009, fid: -0.6790+0.0000j
step: 1010, fid: -0.6155+0.0000j
step: 1011, fid: -0.4861-0.0000j
step: 1012, fid: -0.6192+0.0000j
step: 1013, fid: -0.8241+0.0000j
step: 1014, fid: -0.8341+0.0000j
step: 1015, fid: -0.8982+0.0000j
step: 1016, fid: -0.5306-0.0000j
step: 1017, fid: -0.9119-0.0000j
step: 1018, fid: -0.8339-0.0000j
step: 1019, fid: -0.9397+0.0000j
step: 1020, fid: -0.9066+0.0000j
step: 1021, fid: -0.9067+0.0000j
step: 1022, fid: -0.8851+0.0000j
step: 1023, fid: -0.8259-0.0000j
step: 1024, fid:

step: 1312, fid: -0.9086-0.0000j
step: 1313, fid: -0.8811-0.0000j
step: 1314, fid: -0.7357-0.0000j
step: 1315, fid: -0.7690-0.0000j
step: 1316, fid: -0.6827+0.0000j
step: 1317, fid: -0.7379+0.0000j
step: 1318, fid: -0.8820+0.0000j
step: 1319, fid: -0.7303+0.0000j
step: 1320, fid: -0.7314+0.0000j
step: 1321, fid: -0.8314-0.0000j
step: 1322, fid: -0.9277-0.0000j
step: 1323, fid: -0.9091+0.0000j
step: 1324, fid: -0.6286-0.0000j
step: 1325, fid: -0.7634-0.0000j
step: 1326, fid: -0.8511-0.0000j
step: 1327, fid: -0.8438-0.0000j
step: 1328, fid: -0.9141-0.0000j
step: 1329, fid: -0.8017+0.0000j
step: 1330, fid: -0.8425-0.0000j
step: 1331, fid: -0.9159-0.0000j
step: 1332, fid: -0.9306-0.0000j
step: 1333, fid: -0.6720-0.0000j
step: 1334, fid: -0.9094-0.0000j
step: 1335, fid: -0.8565-0.0000j
step: 1336, fid: -0.9340+0.0000j
step: 1337, fid: -0.6168+0.0000j
step: 1338, fid: -0.5332+0.0000j
step: 1339, fid: -0.8263+0.0000j
step: 1340, fid: -0.5382+0.0000j
step: 1341, fid: -0.8486+0.0000j
step: 1342

...

step: 8380, fid: -0.9522+0.0000j
step: 8381, fid: -0.9486-0.0000j
step: 8382, fid: -0.7658-0.0000j
step: 8383, fid: -0.8103+0.0000j
step: 8384, fid: -0.6947+0.0000j
step: 8385, fid: -0.8195+0.0000j
step: 8386, fid: -0.6288+0.0000j
step: 8387, fid: -0.6305-0.0000j
step: 8388, fid: -0.9263-0.0000j
step: 8389, fid: -0.8776-0.0000j
step: 8390, fid: -0.7690-0.0000j
step: 8391, fid: -0.4381-0.0000j
step: 8392, fid: -0.7679+0.0000j
step: 8393, fid: -0.7685+0.0000j
step: 8394, fid: -0.6631-0.0000j
step: 8395, fid: -0.6654-0.0000j
step: 8396, fid: -0.7944-0.0000j
step: 8397, fid: -0.8088-0.0000j
step: 8398, fid: -0.7039+0.0000j
step: 8399, fid: -0.9463+0.0000j
step: 8400, fid: -0.8829+0.0000j
step: 8401, fid: -0.9465-0.0000j
step: 8402, fid: -0.8386-0.0000j
step: 8403, fid: -0.9472+0.0000j
step: 8404, fid: -0.8238+0.0000j
step: 8405, fid: -0.8485-0.0000j
step: 8406, fid: -0.8483-0.0000j
step: 8407, fid: -0.8330-0.0000j
step: 8408, fid: -0.9321-0.0000j
step: 8409, fid: -0.9500-0.0000j
step: 8410

step: 8721, fid: -0.9311+0.0000j
step: 8722, fid: -0.9403+0.0000j
step: 8723, fid: -0.8375-0.0000j
step: 8724, fid: -0.9013-0.0000j
step: 8725, fid: -0.6325-0.0000j
step: 8726, fid: -0.8536+0.0000j
step: 8727, fid: -0.6268-0.0000j
step: 8728, fid: -0.9473-0.0000j
step: 8729, fid: -0.8929-0.0000j
step: 8730, fid: -0.8366-0.0000j
step: 8731, fid: -0.9324+0.0000j
step: 8732, fid: -0.8319+0.0000j
step: 8733, fid: -0.4617-0.0000j
step: 8734, fid: -0.9446-0.0000j
step: 8735, fid: -0.6148-0.0000j
step: 8736, fid: -0.6351+0.0000j
step: 8737, fid: -0.9006+0.0000j
step: 8738, fid: -0.9384-0.0000j
step: 8739, fid: -0.8384-0.0000j
step: 8740, fid: -0.7364+0.0000j
step: 8741, fid: -0.9400-0.0000j
step: 8742, fid: -0.4964+0.0000j
step: 8743, fid: -0.5737-0.0000j
step: 8744, fid: -0.8476-0.0000j
step: 8745, fid: -0.9420+0.0000j
step: 8746, fid: -0.6858+0.0000j
step: 8747, fid: -0.9468+0.0000j
step: 8748, fid: -0.8494+0.0000j
step: 8749, fid: -0.8381-0.0000j
step: 8750, fid: -0.7207+0.0000j
step: 8751

step: 9063, fid: -0.8476-0.0000j
step: 9064, fid: -0.8904+0.0000j
step: 9065, fid: -0.7911-0.0000j
step: 9066, fid: -0.7964+0.0000j
step: 9067, fid: -0.9285+0.0000j
step: 9068, fid: -0.8913-0.0000j
step: 9069, fid: -0.9300-0.0000j
step: 9070, fid: -0.8388+0.0000j
step: 9071, fid: -0.8819+0.0000j
step: 9072, fid: -0.7155-0.0000j
step: 9073, fid: -0.9507-0.0000j
step: 9074, fid: -0.8008+0.0000j
step: 9075, fid: -0.8552-0.0000j
step: 9076, fid: -0.8792-0.0000j
step: 9077, fid: -0.5145-0.0000j
step: 9078, fid: -0.7069-0.0000j
step: 9079, fid: -0.7080+0.0000j
step: 9080, fid: -0.9329-0.0000j
step: 9081, fid: -0.9106+0.0000j
step: 9082, fid: -0.8789+0.0000j
step: 9083, fid: -0.8320+0.0000j
step: 9084, fid: -0.8329+0.0000j
step: 9085, fid: -0.8416-0.0000j
step: 9086, fid: -0.9299+0.0000j
step: 9087, fid: -0.8897+0.0000j
step: 9088, fid: -0.9511-0.0000j
step: 9089, fid: -0.7303+0.0000j
step: 9090, fid: -0.9263-0.0000j
step: 9091, fid: -0.8934-0.0000j
step: 9092, fid: -0.6839+0.0000j
step: 9093

step: 9405, fid: -0.6914-0.0000j
step: 9406, fid: -0.8091+0.0000j
step: 9407, fid: -0.8259-0.0000j
step: 9408, fid: -0.7386+0.0000j
step: 9409, fid: -0.8761+0.0000j
step: 9410, fid: -0.8763-0.0000j
step: 9411, fid: -0.8585+0.0000j
step: 9412, fid: -0.8775-0.0000j
step: 9413, fid: -0.8260-0.0000j
step: 9414, fid: -0.9482+0.0000j
step: 9415, fid: -0.8321+0.0000j
step: 9416, fid: -0.6322-0.0000j
step: 9417, fid: -0.9462-0.0000j
step: 9418, fid: -0.5921-0.0000j
step: 9419, fid: -0.6956-0.0000j
step: 9420, fid: -0.9127-0.0000j
step: 9421, fid: -0.8531+0.0000j
step: 9422, fid: -0.6901-0.0000j
step: 9423, fid: -0.8072+0.0000j
step: 9424, fid: -0.9374-0.0000j
step: 9425, fid: -0.5675-0.0000j
step: 9426, fid: -0.8452-0.0000j
step: 9427, fid: -0.8472-0.0000j
step: 9428, fid: -0.8538+0.0000j
step: 9429, fid: -0.9381+0.0000j
step: 9430, fid: -0.9046+0.0000j
step: 9431, fid: -0.9492+0.0000j
step: 9432, fid: -0.9494+0.0000j
step: 9433, fid: -0.8080+0.0000j
step: 9434, fid: -0.8380-0.0000j
step: 9435

step: 9746, fid: -0.6331-0.0000j
step: 9747, fid: -0.6219-0.0000j
step: 9748, fid: -0.8560+0.0000j
step: 9749, fid: -0.9268-0.0000j
step: 9750, fid: -0.6771+0.0000j
step: 9751, fid: -0.7969-0.0000j
step: 9752, fid: -0.9071+0.0000j
step: 9753, fid: -0.8185+0.0000j
step: 9754, fid: -0.8344+0.0000j
step: 9755, fid: -0.9283+0.0000j
step: 9756, fid: -0.5320+0.0000j
step: 9757, fid: -0.8130+0.0000j
step: 9758, fid: -0.5345-0.0000j
step: 9759, fid: -0.4258+0.0000j
step: 9760, fid: -0.8394+0.0000j
step: 9761, fid: -0.5241+0.0000j
step: 9762, fid: -0.9241-0.0000j
step: 9763, fid: -0.8420-0.0000j
step: 9764, fid: -0.8432+0.0000j
step: 9765, fid: -0.8892+0.0000j
step: 9766, fid: -0.8074-0.0000j
step: 9767, fid: -0.8164+0.0000j
step: 9768, fid: -0.8921-0.0000j
step: 9769, fid: -0.6430+0.0000j
step: 9770, fid: -0.8857-0.0000j
step: 9771, fid: -0.9374+0.0000j
step: 9772, fid: -0.8764-0.0000j
step: 9773, fid: -0.7684-0.0000j
step: 9774, fid: -0.8865-0.0000j
step: 9775, fid: -0.9080-0.0000j
step: 9776

In [6]:
kraus_list_target[0].T.conj()@kraus_list_target[0] + kraus_list_target[1].T.conj()@kraus_list_target[1]

tensor([[ 1.00000000e+00+0.00000000e+00j, -2.08166817e-17-1.51788304e-17j,
         -1.21430643e-17-2.77555756e-17j, -7.63278329e-17+3.46944695e-18j,
          0.00000000e+00+1.38777878e-17j,  0.00000000e+00+1.38777878e-17j,
          0.00000000e+00+6.93889390e-17j,  0.00000000e+00+0.00000000e+00j],
        [-2.08166817e-17+2.19008839e-17j,  1.00000000e+00+0.00000000e+00j,
          6.93889390e-17+1.66533454e-16j,  4.16333634e-17-5.55111512e-17j,
         -1.11022302e-16+5.55111512e-17j,  8.32667268e-17+3.12250226e-17j,
         -8.32667268e-17-8.32667268e-17j,  1.80411242e-16+1.11022302e-16j],
        [-1.21430643e-17+0.00000000e+00j,  6.93889390e-17-1.80411242e-16j,
          1.00000000e+00+0.00000000e+00j,  6.93889390e-17+4.16333634e-17j,
          1.11022302e-16-6.93889390e-17j, -6.24500451e-17+2.77555756e-17j,
          4.68375339e-17+0.00000000e+00j, -1.38777878e-17+6.93889390e-17j],
        [-7.63278329e-17+0.00000000e+00j,  4.16333634e-17+1.38777878e-17j,
          6.93889390e-

In [23]:
n = 2
d = 2**n
rank = 2

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(42)
torch.manual_seed(42)

X_target, _, _ = generate_ginibre(rank*d, rank*d)
kraus_list_target, U, Q, R = generate_kraus(X_target, d, rank)

In [24]:
print(torch.diag(R))

tensor([-2.21147645+0.j, -3.93841711+0.j,  3.78146082+0.j, -3.12663678+0.j,
         2.12048085+0.j,  2.57248815+0.j, -2.04652399+0.j,  0.48867031+0.j],
       dtype=torch.complex128)


In [25]:
print(Q@Q.conj().T)

tensor([[ 1.00000000e+00+0.00000000e+00j,  2.08166817e-17+2.77555756e-17j,
         -2.94902991e-17-4.16333634e-17j,  3.46944695e-18-1.28369537e-16j,
          1.24900090e-16-1.14491749e-16j,  1.97758476e-16+1.38777878e-17j,
         -1.87350135e-16-2.77555756e-17j, -3.08997619e-17-1.21430643e-16j],
        [ 2.08166817e-17-5.55111512e-17j,  1.00000000e+00+0.00000000e+00j,
          9.02056208e-17-8.32667268e-17j,  8.32667268e-17-2.77555756e-17j,
         -2.77555756e-17+5.55111512e-17j, -1.38777878e-17+4.16333634e-17j,
         -4.16333634e-17+0.00000000e+00j, -5.72458747e-17+0.00000000e+00j],
        [-2.94902991e-17+5.55111512e-17j,  9.02056208e-17+4.51028104e-17j,
          1.00000000e+00+0.00000000e+00j,  1.35308431e-16-5.03069808e-17j,
          6.24500451e-17-2.77555756e-17j,  2.77555756e-17-7.63278329e-17j,
          4.85722573e-17-4.85722573e-17j,  2.62376926e-17+4.85722573e-17j],
        [ 3.46944695e-18+1.11022302e-16j,  8.32667268e-17+1.38777878e-17j,
          1.35308431e-

In [28]:
print((U@U.conj().T).shape)

torch.Size([8, 8])


In [29]:
print(U.conj().T@U)

tensor([[ 1.00000000e+00+0.00000000e+00j,  5.55111512e-17-5.55111512e-17j,
         -2.08166817e-17+0.00000000e+00j,  0.00000000e+00+2.77555756e-17j,
          4.51028104e-17+5.55111512e-17j, -4.16333634e-17+1.73472348e-17j,
          1.11022302e-16+2.77555756e-17j, -1.50053581e-16+6.93889390e-18j],
        [ 5.55111512e-17+6.93889390e-17j,  1.00000000e+00+0.00000000e+00j,
         -1.56125113e-17-3.98986399e-17j, -1.38777878e-17+9.02056208e-17j,
         -8.67361738e-18-4.16333634e-17j, -9.71445147e-17+8.93382590e-17j,
         -1.38777878e-17-2.60208521e-17j, -2.97071395e-17-2.77555756e-17j],
        [-2.08166817e-17+0.00000000e+00j, -1.56125113e-17+3.98986399e-17j,
          1.00000000e+00+0.00000000e+00j,  4.85722573e-17-2.08166817e-17j,
          5.89805982e-17+3.46944695e-17j,  3.46944695e-17+5.72458747e-17j,
          2.08166817e-17-1.56125113e-17j, -8.27246258e-17-1.38777878e-17j],
        [ 0.00000000e+00-2.77555756e-17j, -1.38777878e-17-7.28583860e-17j,
          4.85722573e-

In [11]:
print(U@U.conj().T)

tensor([[ 1.00000000e+00+0.00000000e+00j,  2.08166817e-17+2.77555756e-17j,
         -2.94902991e-17-4.16333634e-17j,  3.46944695e-18-1.28369537e-16j,
          1.24900090e-16-1.14491749e-16j,  1.97758476e-16+1.38777878e-17j,
         -1.87350135e-16-2.77555756e-17j, -3.08997619e-17-1.21430643e-16j],
        [ 2.08166817e-17-5.55111512e-17j,  1.00000000e+00+0.00000000e+00j,
          9.02056208e-17-8.32667268e-17j,  8.32667268e-17-2.77555756e-17j,
         -2.77555756e-17+5.55111512e-17j, -1.38777878e-17+4.16333634e-17j,
         -4.16333634e-17+0.00000000e+00j, -5.72458747e-17+0.00000000e+00j],
        [-2.94902991e-17+5.55111512e-17j,  9.02056208e-17+4.51028104e-17j,
          1.00000000e+00+0.00000000e+00j,  1.35308431e-16-5.03069808e-17j,
          6.24500451e-17-2.77555756e-17j,  2.77555756e-17-7.63278329e-17j,
          4.85722573e-17-4.85722573e-17j,  2.62376926e-17+4.85722573e-17j],
        [ 3.46944695e-18+1.11022302e-16j,  8.32667268e-17+1.38777878e-17j,
          1.35308431e-